In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KDTree
import os
import glob
import seaborn as sns
import math


# Create a dictionary with key = eid, value = dataframe from each experiment. 
directory = r""
analysis_directory = directory + "/analysis" 
if not os.path.exists(analysis_directory):
    os.makedirs(analysis_directory)
add_to_filename = "ASH_nlsGC6m"              
all_files = glob.glob(directory + "/*.csv")
file_list = [] 
df_raw = {} # Dictionary of keys = file name, values = dataframe of raw csv.

total_sec           = 25
stim_on_sec         = 5 
stim_off_sec        = 15
frames_per_sec      = 10
frame_rate          = 1/frames_per_sec
frames              = np.arange(0, total_sec*frames_per_sec)


def smooth (x,window_len=11,window='hanning'):
        s = np.r_[2*x[0]-x[window_len-1::-1],x,2*x[-1]-x[-1:-window_len:-1]]
        if window == 'flat': #moving average
                w=np.ones(window_len,'d')
        else:  
                w=eval('np.'+window+'(window_len)')
        y = np.convolve(w/w.sum(),s,mode='same')
        return y[window_len:-window_len+1]

## Go through each file in the directory to get data
for i, file_name in enumerate (all_files):
    file_name_cut = file_name[(len(directory)+1):-4]
    file_list.append(file_name_cut)

    df_raw[file_name_cut] = pd.read_csv(file_name, index_col=None, header=0)
    
    neuron          = np.array(df_raw[file_name_cut]["Signal"])
    background      = np.array(df_raw[file_name_cut]["BG"])
    neuron          = neuron[:250]
    background      = background[:250]   
    subtract        = neuron - background
    F0              = np.median(subtract[40:49])  
    dFF0            = (subtract - F0)/F0*100
    
    ## change to list 
    frame_list  = list(frames)    
    dFF0        = list(dFF0)
    sm_dFF0     = list(sm_dFF0)
    
    ## peak time is where the dFF0 is max after smoothing by 21.   
    sm_dFF0_peak   = max(sm_dFF0) 
    peak_index     = sm_dFF0.index(sm_dFF0_peak)
    time_peak      = frame_list[peak_index]/frames_per_sec
                     
## Re-arrange the dataframe according to the peak value
df_dFF0_peak_sorted = df_dFF0_peak.sort_values(ascending = False)

for i, file_name_cut in enumerate(df_dFF0_peak_sorted.index):
    if i ==0:
        df_dFF0_heatmap_sorted = pd.DataFrame(df_dFF0[file_name_cut], columns = [file_name_cut])
        df_dFF0_scaled_sorted  = pd.DataFrame(df_dFF0_scaled[file_name_cut], columns = [file_name_cut])
    else:
        df_dFF0_heatmap_sorted[file_name_cut] = pd.Series(df_dFF0[file_name_cut])
        df_dFF0_scaled_sorted[file_name_cut]  = pd.Series(df_dFF0_scaled[file_name_cut])
      
    
df_dFF0_heatmap_sorted = df_dFF0_heatmap_sorted.T      
df_dFF0_scaled_sorted  = df_dFF0_scaled_sorted.T

## Heatmap same scale
ax1 = plt.axes()
sns.heatmap(df_dFF0_heatmap_sorted, xticklabels = 50, yticklabels=True, cmap="jet", vmax=500, vmin=-50, ax = ax1)
bottom, top = ax1.get_ylim()
ax1.set_ylim(bottom + 0.5, top -0.5)
ax1.set_title("%s dF/F0 n = %d" % (add_to_filename, (i+1)))
save_file_path = analysis_directory + "/" + add_to_filename + "_heatmap_same_scale.png"
figure = ax1.get_figure() 
figure.savefig(save_file_path, dpi=1200)
plt.show()

# Calculate mean 
df_dFF0_values = df_dFF0.drop(columns=["frame"])
dFF0_mean      = df_dFF0_values.mean(axis=1)
dFF0_stderr    = df_dFF0_values.std(axis=1)
dFF0_stderr    = dFF0_stderr/math.sqrt(len(all_files))

## Figure of mean trace with standard error, same scale
plt.figure(figsize = (10,6))
plt.errorbar(frames/frames_per_sec, dFF0_mean, dFF0_stderr, linewidth = 2, color = "black", ecolor = "lightgray")    
plt.ylim(-20, 250)
plt.xlabel("time (sec)")
plt.ylabel("deltaF/F0 (%)")
plt.title("%s mean trace" % add_to_filename)
save_file_path = analysis_directory + "/" + add_to_filename + "_mean_stderr_dFF0_same_scale.png"
plt.savefig(save_file_path)
plt.show() 
